## Setup Only for Colab

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/hidden_mediators

In [ ]:
%ls

In [ ]:
from IPython.display import clear_output

In [ ]:
import time
!pip install -r requirements.txt
time.sleep(2)
clear_output()

In [ ]:
import time
# replace `develop` with `install` if you wont make library code changes
!python setup.py develop
time.sleep(2)
clear_output()
# Restart the session after running this

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks

# Main Logic

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
from joblib import Parallel, delayed
from proximalde.gen_data import gen_data_complex, gen_data_no_controls
from proximalde.proximal import proximal_direct_effect, ProximalDE, residualizeW
from sklearn.linear_model import LinearRegression
from proximalde.crossfit import fit_predict

# Running a Single Experiment

In [ ]:
def exp_res(it, n, pw, pz, px, a, b, c, d, e, f, g, *,
            dual_type='Z', n_splits=5, semi=False,
            multitask=False, n_jobs=-1, verbose=0):
    np.random.seed(it)
    # M is unobserved so we omit it from the return variables
    W, D, _, Z, X, Y = gen_data_complex(n, pw, pz, px, a, b, c, d, e, f, g)
    # return fit_predict(X, Y, LinearRegression(), LinearRegression(), n_splits, semi, multitask, n_jobs, 3)
    # return residualizeW(W, D, Z, X, Y)
    return proximal_direct_effect(W, D, Z, X, Y,
                                  dual_type=dual_type,
                                  cv=n_splits, semi=semi,
                                  multitask=multitask, n_jobs=n_jobs,
                                  verbose=verbose,
                                  random_state=it)

In [ ]:
a = 1.0  # a*b is the indirect effect through mediator
b = 1.0
c = .5  # this is the direct effect we want to estimate
d = .6  # this can be zero; does not hurt
e = .7  # if the product of e*f is small, then we have a weak instrument
f = .5  # if the product of e*f is small, then we have a weak instrument
g = .9  # this can be zero; does not hurt

In [ ]:
n = 100000
pw = 100
pz, px = 170, 60
n_splits = 3

In [ ]:
point, stderr, r2D, r2Z, r2X, r2Y, idstrength, idstrength_std, point_pre, stderr_pre = exp_res(3, n, pw, pz, px, a, b, c, d, e, f, g,
                                                 dual_type='Z', n_splits=n_splits, semi=True, verbose=1)
pd.DataFrame({'point': point, 'se': stderr, 'r2D': r2D, 'r2Z': r2Z, 'r2X': r2X, 'r2Y': r2Y, 'Jacobian': idstrength,
              'point_pre': point_pre, 'se_pre': stderr_pre}, index=[0])

### Using the ProximalDE Estimator Class

In [ ]:
np.random.seed(4)
W, D, _, Z, X, Y = gen_data_complex(n, pw, pz, px, a, b, c, d, e, f, g)
# W, D, _, Z, X, Y = gen_data_no_controls(n, pw, pz, px, a, b, c, d, e, f, g)
est = ProximalDE(cv=3, semi=True,
                 multitask=False, n_jobs=-1, random_state=3, verbose=3)
est.fit(W, D, Z, X, Y)

In [ ]:
est.summary(decimals=5)

#### Covariance Rank Diagnostic for Covariance of Proxies

In [ ]:
svalues, svalues_crit = est.covariance_rank_test(calculate_critical=True)

In [ ]:
plt.title(f"Number of singular values above threshold: {np.sum(svalues >= svalues_crit)}. "
          f"Threshold={svalues_crit:.3f}. Top singular value={svalues[0]:.3f}")
plt.scatter(np.arange(len(svalues)), svalues)
plt.axhline(svalues_crit)
plt.show()

#### Confidence Intervals and Robust Confidence Intervals

In [ ]:
est.conf_int(alpha=.05) # 95% confidence interval

In [ ]:
# 95% confidence interval, robust to weak identification
est.robust_conf_int(alpha=0.05, lb=.1, ub=1.0, ngrid=1000)

#### Unusual Data Diagnostics

In [ ]:
diag = est.run_diagnostics()

In [ ]:
inds = est.influential_set(alpha=0.05)
len(inds)  # size of influential set that can flip the result

In [ ]:
from sklearn.base import clone
# let's re-train a clone of the estimator on all the data
# except the influential set
est2 = clone(est)
est2.fit(np.delete(W, inds, axis=0), np.delete(D, inds, axis=0),
         np.delete(Z, inds, axis=0), np.delete(X, inds, axis=0),
         np.delete(Y, inds, axis=0))
est2.summary(alpha=0.05)

In [ ]:
diag.cookd_plot()
plt.show()

In [ ]:
diag.l2influence_plot()
plt.show()

In [ ]:
diag.influence_plot(influence_measure='cook', npoints=10)
plt.show()

In [ ]:
diag.influence_plot(influence_measure='l2influence', npoints=10)
plt.show()

### Subsample-Based Inference

In [ ]:
inf = est.bootstrap_inference(stage=3, n_subsamples=1000, fraction=0.5, replace=False, verbose=3, random_state=123)
inf.summary()

In [ ]:
plt.hist(inf.point_dist)
plt.vlines([inf.point], 0, 300, color='r')
plt.show()

In [ ]:
inf = est.bootstrap_inference(stage=2, n_subsamples=100, fraction=0.5, replace=False, verbose=3, random_state=123)
inf.summary()

In [ ]:
plt.hist(inf.point_dist)
plt.vlines([inf.point], 0, 300, color='r')
plt.show()

In [ ]:
inf = est.bootstrap_inference(stage=1, n_subsamples=10, fraction=0.5, replace=False, verbose=3, random_state=123)
inf.summary()

In [ ]:
plt.hist(inf.point_dist)
plt.vlines([inf.point], 0, 300, color='r')
plt.show()

In [ ]:
inf.summary(pivot=True)

# Coverage Experiment

In [ ]:
def exp_res(it, n, pw, pz, px, a, b, c, d, e, f, g, *,
            dual_type='Z', ivreg_type='adv', n_splits=5, semi=False,
            multitask=False, n_jobs=-1, verbose=0):
    np.random.seed(it)
    if pw > 0:
        # M is unobserved so we omit it from the return variables
        W, D, _, Z, X, Y = gen_data_complex(n, pw, pz, px, a, b, c, d, e, f, g)
        est = ProximalDE(cv=n_splits, semi=semi,
                         dual_type=dual_type, ivreg_type=ivreg_type,
                         multitask=multitask, n_jobs=n_jobs,
                         random_state=it, verbose=verbose)
        est.fit(W, D, Z, X, Y)
    else:
        _, D, _, Z, X, Y = gen_data_no_controls(n, pw, pz, px, a, b, c, d, e, f, g)
        est = ProximalDE(cv=n_splits, semi=semi,
                         dual_type=dual_type, ivreg_type=ivreg_type,
                         multitask=multitask, n_jobs=n_jobs,
                         random_state=it, verbose=verbose)
        est.fit(None, D, Z, X, Y)
    return est.point_, est.stderr_, est.r2D_, est.r2Z_, est.r2X_, est.r2Y_, \
        est.idstrength_, est.idstrength_std_, est.point_pre_, est.stderr_pre_

In [ ]:
a = 1.0  # a*b is the indirect effect through mediator
b = 1.0
c = .5  # this is the direct effect we want to estimate
d = .6  # this can be zero; does not hurt
e = .7  # if the product of e*f is small, then we have a weak instrument
f = .5  # if the product of e*f is small, then we have a weak instrument
g = .9  # this can be zero; does not hurt
n = 100000
pw = 0
pz, px = 20, 20

results = Parallel(n_jobs=-1, verbose=3)(delayed(exp_res)(i, n, pw, pz, px, a, b, c, d, e, f, g,
                                                          dual_type='Z', ivreg_type='adv',
                                                          n_splits=3, semi=True, n_jobs=1)
                                          for i in range(100))

#### Summarize

In [ ]:
points_base, stderrs_base, rmseD, rmseZ, rmseX, rmseY, strength, strength_std, points_alt, stderrs_alt = zip(*results)

points_base = np.array(points_base)
stderrs_base = np.array(stderrs_base)
points_alt = np.array(points_alt)
stderrs_alt = np.array(stderrs_alt)

for points, stderrs in [(points_base, stderrs_base), (points_alt, stderrs_alt)]:
    coverage = np.mean((points + 1.96 * stderrs >= c) & (points - 1.96 * stderrs <= c))
    rmse = np.sqrt(np.mean((points - c)**2))
    bias = np.abs(np.mean(points) - c)
    std = np.std(points)
    mean_stderr = np.mean(stderrs)
    mean_length = np.mean(2 * 1.96 * stderrs)
    median_length = np.median(2 * 1.96 * stderrs)
    print(f"Coverage: {coverage:.3f}")
    print(f"RMSE: {rmse:.3f}")
    print(f"Bias: {bias:.3f}")
    print(f"Std: {std:.3f}")
    print(f"Mean CI length: {mean_length:.3f}")
    print(f"Median CI length: {mean_length:.3f}")
    print(f"Mean Estimated Stderr: {mean_stderr:.3f}")
    print(f"Nuisance R^2 (D, Z, X, Y): {np.mean(rmseD):.3f}, {np.mean(rmseZ):.3f}, {np.mean(rmseX):.3f}, {np.mean(rmseY):.3f}")

In [ ]:
plt.hist(points_base, label='Distribution of Estimates: debiased')
plt.hist(points_alt, label='Distribution of Estimates: original', alpha=.3)
plt.vlines([c], 0, plt.ylim()[1], color='red', label='truth')
plt.legend()
plt.show()